# Language Modeling using Ngram

In this Exercise, we are going to create a bigram language model and its variation. We will build one model for each of the following type and calculate their perplexity:
- Unigram Model
- Bigram Model
- Bigram Model with Laplace smoothing
- Bigram Model with Interpolation
- Bigram Model with Kneser-ney Interpolation

We will also use NLTK which is a natural language processing library for python to make our lives easier.



In [ ]:
# #download corpus
!wget --no-check-certificate https://github.com/ekapolc/nlp_2019/raw/master/HW4/BEST2010.zip
!unzip BEST2010.zip

In [ ]:
!wget https://www.dropbox.com/s/jajdlqnp5h0ywvo/tokenized_wiki_sample.csv

In [ ]:
#First we import necessary library such as math, nltk, bigram, and collections.
import math
import nltk
import io
import random
from random import shuffle
from nltk import bigrams, trigrams
from collections import Counter, defaultdict
random.seed(999)

BEST2010 is a free Thai NLP dataset by NECTEC usually used as a standard benchmark for various NLP tasks including language modeling. It is separated into 4 domains including article, encyclopedia, news, and novel. The data is already  tokenized using '|' as a separator.

For example,

ตาม|ที่|นางประนอม ทองจันทร์| |กับ| |ด.ช.กิตติพงษ์ แหลมผักแว่น| |และ| |ด.ญ.กาญจนา กรองแก้ว| |ป่วย|สงสัย|ติด|เชื้อ|ไข้|ขณะ|นี้|ยัง|ไม่|ดี|ขึ้น|

In [ ]:
total_word_count = 0
best2010 = []
with open('BEST2010/news.txt','r',encoding='utf-8') as f:
  for i,line in enumerate(f):
    line=line.strip()[:-1] #remove the trailing |
    total_word_count += len(line.split("|"))
    best2010.append(line)

In [ ]:
#For simplicity, we assumes that each line is a sentence.
print (f'Total sentences in BEST2010 news dataset :\t{len(best2010)}')
print (f'Total word counts in BEST2010 news dataset :\t{total_word_count}')

We separate the input into 2 sets, train and test data with 70:30 ratio

In [ ]:
sentences = best2010
# The data is separated to train and test set with 70:30 ratio.
train = sentences[:int(len(sentences)*0.7)]
test = sentences[int(len(sentences)*0.7):]

#Training data
train_word_count =0
for line in train:
    for word in line.split('|'):
        train_word_count+=1
print ('Total sentences in BEST2010 news training dataset :\t'+ str(len(train)))
print ('Total word counts in BEST2010 news training dataset :\t'+ str(train_word_count))

Here we load the data from Wikipedia which is also already tokenized. It will be used for answering questions in MyCourseville.

In [ ]:
import pandas as pd
wiki_data = pd.read_csv("tokenized_wiki_sample.csv")

## Data Preprocessing

Before training any language models, the first step we always do is process the data into the format suited for the LM.

For this exercise, we will use NLTK to help process our data.

In [ ]:
from nltk.lm.preprocessing import pad_both_ends, flatten
from nltk.lm.vocabulary import Vocabulary
from nltk import ngrams

We begin by "tokenizing" our training set. Note that the data is already tokenized so we can just split it.

In [ ]:
tokenized_train = [["<s>"] + t.split("|") + ["</s>"] for t in train] # "tokenize" and pad each sentence

Next we create a vocabulary with the ```Vocabulary``` class from NLTK. It accepts a list of tokens so we flatten our sentences into one long sentence first.







In [ ]:
flat_tokens = list(flatten(tokenized_train)) #join all sentences into one long sentence
vocab = Vocabulary(flat_tokens, unk_cutoff=3) #Words with frequency **below** 3 (not exactly 3) will not be considered in our vocab and will be converted to <UNK>.

Then we replace low frequency words.

Now *each* sentence is going to look something like this:
\["\<s\>", "hello", "my", "name", "is", "\<UNK\>", "\</s\>" \]

In [ ]:
tokenized_train = [[token if token in vocab else "<UNK>" for token in sentence] for sentence in tokenized_train]

Finally, we do the same for the test set and the wiki dataset.

In [ ]:
tokenized_test = [t.split("|") for t in test]
tokenized_test = [[token if token in vocab else "<UNK>" for token in sentence] for sentence in tokenized_test]

tokenized_wiki_test = [t.split("|") for t in wiki_data['tokenized'].tolist()]
tokenized_wiki_test = [[token if token in vocab else "<UNK>" for token in sentence] for sentence in tokenized_wiki_test]

# Unigram

In this section, we will demonstrate how to build a unigram language model <br>
**Important note:** <br>
**\<s\>** = sentence start symbol <br>
**\</s\>** = sentence end symbol

# VERY IMPORTANT:
- In this notebook, we will *not* default the unknown token probability to ```1/len(vocab)``` but instead will treat it as a normal word and let the model learn its probability so that we can compare our results to NLTK.
- **Also make sure that the code in this notebook can be executed without any problem. If we find that you used NLTK to answer questions in MyCourseVille and did not finish the assignment, you will receive a grade of 0 for this assignment.**

In [ ]:
class UnigramModel():
  def __init__(self, data, vocab):
    self.unigram_count = defaultdict(lambda: 0.0)
    self.word_count = 0
    self.vocab = vocab
    for sentence in data:
        for w in sentence: #[(word1, ), (word2, ), (word3, )...]
          w = w[0]
          self.unigram_count[w] +=1.0
          self.word_count+=1

  def __getitem__(self, w):
    w = w[0]  #[(word1, ), (word2, ), (word3, )...]
    if w in self.vocab:
      return self.unigram_count[w]/(self.word_count)
    else:
      return self.unigram_count["<UNK>"]/(self.word_count)

In [ ]:
train_unigrams = [list(ngrams(sent, n=1)) for sent in tokenized_train] #creating the unigrams by setting n=1
model = UnigramModel(train_unigrams, vocab)

In [ ]:
def getLnValue(x):
      if x == 0:
        return -math.inf
      return math.log(x)

In [ ]:
#problability of 'นายก'
print(getLnValue(model[('นายก',)]))

#for example, problability of 'นายกรัฐมนตรี' which is an unknown word is equal to
print(getLnValue(model[('นายกรัฐมนตรี',)]))

#problability of 'นายก' 'ได้' 'ให้' 'สัมภาษณ์' 'กับ' 'สื่อ'
prob = getLnValue(model[('นายก',)])+getLnValue(model[('ได้',)])+ getLnValue(model[('ให้',)])+getLnValue(model[('สัมภาษณ์',)])+getLnValue(model[('กับ',)])+getLnValue(model[('สื่อ',)])+getLnValue(model[('</s>',)])
print ('Problability of a sentence', math.exp(prob))

# Perplexity

In order to compare language model we need to calculate perplexity. In this task you should write a perplexity calculation code for the unigram model. The result perplexity should be around 448.90 and
392.74 on train and test data.

## TODO #1 Calculate perplexity

In [ ]:
def getLnValue(x):
    pass

def calculate_sentence_ln_prob(sentence, model):
    pass

def perplexity(test,model):
    pass

In [ ]:
test_unigrams = [list(ngrams(sent, n=1)) for sent in tokenized_test]

In [ ]:
print(perplexity(train_unigrams,model))
print(perplexity(test_unigrams,model))

## Q1 MCV
Calculate the perplexity of the model on the wiki test set and answer in MyCourseVille

In [ ]:
wiki_test_unigrams = [list(ngrams(sent, n=1)) for sent in tokenized_wiki_test]

In [ ]:
print(perplexity([list(flatten(wiki_test_unigrams))], model))

# Bigram

Next, you will create a better language model than a unigram (which is not much to compare with). But first, it is very tedious to count every pair of words that occur in our corpus by ourselves. Lucky for us, nltk provides us a simple library which will simplify the process.

In [ ]:
#example of nltk usage for bigram
sentence = 'I always search google for an answer .'
padded_sentence = list(pad_both_ends(sentence.split(), n=2))

print('This is how nltk generate bigram.')
for w1,w2 in bigrams(padded_sentence):
    print(w1,w2)
print('\n<s> and </s> are used as a start and end of sentence symbol. respectively.')

Now, you should be able to implement a bigram model by yourself. Also, you must create a new perplexity calculation for bigram. The result perplexity should be around 56.46 and inf on train and test data.

## TODO #3 Write Bigram Model

In [ ]:
class BigramModel():
  def __init__(self, data, vocab):
    pass

  def __getitem__(self, bigram):
    w1, w2 = bigram
    pass

## TODO #4 Write Perplexity for Bigram Model

Sum perplexity score at a sentence level, instead of word level

In [ ]:
def perplexity(bigram_data,model):
  pass

In [ ]:
train_bigrams = [list(ngrams(sent, n=2)) for sent in tokenized_train]
test_bigrams = [list(ngrams(sent, n=2)) for sent in tokenized_test]

In [ ]:
bigram_model_scratch = BigramModel(train_bigrams, vocab)

In [ ]:
print(perplexity([list(flatten(train_bigrams))], bigram_model_scratch))
print(perplexity([list(flatten(test_bigrams))[:16]], bigram_model_scratch)) #can be used to compare with nltk
print(perplexity([list(flatten(test_bigrams))], bigram_model_scratch))

## Q2 MCV

In [ ]:
wiki_test_bigrams = [list(ngrams(sent, n=2)) for sent in tokenized_wiki_test]

In [ ]:
print(perplexity([list(flatten(wiki_test_bigrams))],bigram_model_scratch))

# Smoothing

Usually any ngram models have a sparsity problem, which means it does not have every possible ngram of words in the dataset. Smoothing techniques can alleviate this problem. In this section, you will implement three basic smoothing methods laplace smoothing, interpolation for bigram, and Knesey-Ney smoothing.

## TODO #5 write Bigram with Laplace smoothing (Add-One Smoothing)

The result perplexity on training and testing should be:

    370.28, 369.16 for Laplace smoothing

In [ ]:
class BigramWithLaplaceSmoothing():

  def __init__(self, data, vocab):
    pass

  def __getitem__(self, bigram):
    pass

model = BigramWithLaplaceSmoothing(train_bigrams, vocab)
print(perplexity([list(flatten(train_bigrams))],model))
print(perplexity([list(flatten(test_bigrams))], model))

## Q3 MCV

In [ ]:
print(perplexity([list(flatten(wiki_test_bigrams))],model))

## TODO #6 Write Bigram with Interpolation
Set the lambda value as 0.7 for bigram, 0.25 for unigram, and 0.05 for unknown word.

The result perplexity on training and testing should be:

    70.07, 102.99 for Interpolation

In [ ]:
class BigramWithInterpolation():

  def __init__(self, data, vocab):
    pass

  def __getitem__(self, bigram):
      pass

model = BigramWithInterpolation(train_bigrams, vocab)
print(perplexity([list(flatten(train_bigrams))],model))
print(perplexity([list(flatten(test_bigrams))], model))

## Q4 MCV

In [ ]:
print(perplexity([list(flatten(wiki_test_bigrams))],model))

## Language modeling on multiple domains

Sometimes, we do not have enough data to create a language model for a new domain. In that case, we can improvised by combining several models to improve result on the new domain.

In this exercise you will try to merge two language models from news and article domains to create a language model for the encyclopedia domain.

In [ ]:
# create encyclopeida data (test data)
encyclo_data=[]
with open('BEST2010/encyclopedia.txt','r',encoding='utf-8') as f:
    for i,line in enumerate(f):
        encyclo_data.append(line.strip()[:-1])

(news) First, you should try to calculate perplexity of your bigram with interpolation on encyclopedia data. The  perplexity should be around 236.33

In [ ]:
encyclopedia_bigrams = ...

In [ ]:
# 1) news only on "encyclopedia"
print(perplexity([list(flatten(encyclopedia_bigrams))], model))

## TODO #7 - Langauge Modelling on Multiple Domains
Combine news and article datasets to create another bigram model and evaluate it on the encyclopedia data.



(article) For your information, a bigram model with interpolation using article data to test on encyclopedia data has a perplexity of 218.57

In [ ]:
# 2) article only on "encyclopedia"
best2010_article=[]
with open('BEST2010/article.txt','r',encoding='utf-8') as f:
    for i,line in enumerate(f):
        best2010_article.append(line.strip()[:-1])

combined_total_word_count = 0
for line in best2010_article:
    combined_total_word_count += len(line.split('|'))

article_bigrams = ...
article_vocab = ...

model = BigramWithInterpolation(article_bigrams, article_vocab)

In [ ]:
print('Perplexity of the bigram model using article data with interpolation smoothing on encyclopedia test data',perplexity([list(flatten(encyclopedia_bigrams))], model))

In [ ]:
# 3) train on news + article, test on "encyclopedia"
best2010_article_and_news = best2010_article.copy()
with open('BEST2010/news.txt','r',encoding='utf-8') as f:
    for i,line in enumerate(f):
        best2010_article_and_news.append(line.strip()[:-1])

combined_bigrams = ...
combined_vocab = ...

combined_model = BigramWithInterpolation(combined_bigrams, combined_vocab)
print('Perplexity of the combined Bigram model with interpolation smoothing on encyclopedia test data',perplexity([list(flatten(encyclopedia_bigrams))], combined_model))

## TODO #8 - Kneser-ney on "News"

<!-- Reimplement equation 4.33 in SLP textbook (https://lagunita.stanford.edu/c4x/Engineering/CS-224N/asset/slp4.pdf) -->

Implement Bigram Knerser-ney LM. The result perplexity should be around 65.81, 93.21 on train and test data. Be careful not to mix up vocab from the above section!


In [ ]:
class BigramKneserNey():

  def __init__(self, data, vocab):
    pass


  def __getitem__(self, x):
    pass

model = BigramKneserNey(train_bigrams, vocab)
print(perplexity([list(flatten(train_bigrams))],model))
print(perplexity([list(flatten(train_bigrams))[:1000]],model)) #can be used to compare with nltk
print(perplexity([list(flatten(test_bigrams))[:1000]], model)) #can be used to compare with nltk
print(perplexity([list(flatten(test_bigrams))], model))

## Q5 MCV

In [ ]:
print(perplexity([list(flatten(wiki_test_bigrams))],model))